# Crime Statistics Resampling Techniques

### Import Dependencies

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

### Read the CSV and Perform Basic Data Cleaning

In [3]:
# Load the data
df = pd.read_csv('../Resources/CSV/crime_data.csv')

In [4]:
# Drop the year column to not skew data with year count and averages
df = df.drop(columns =['Year'])
df.head()

,County,Agency_Count,Murder,Rape,Assault,Burglary,Larceny,Auto_Theft,Violent_Offenses,NonViolent_Offenses,Total_Crime,Population
0,Austin County,5,0,8,53,121,283,13,61,417,482,29354
1,Austin County,5,0,6,43,136,239,31,49,406,466,29718
2,Austin County,5,0,11,34,98,183,45,45,326,376,29963
3,Austin County,5,0,11,32,82,138,29,43,249,295,29912
4,Austin County,5,0,5,38,80,147,29,43,256,302,30009


In [5]:
# Convert the target column values to violent and nonviolent based on their crime
x = dict.fromkeys(['Murder', 'Rape', 'Assault'], 'violent_crimes')    
df = df.replace(x)

x = dict.fromkeys(['Burglary', 'Larceny', 'Auto_Theft'], 'nonviolent_crimes')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,County,Agency_Count,Murder,Rape,Assault,Burglary,Larceny,Auto_Theft,Violent_Offenses,NonViolent_Offenses,Total_Crime,Population
0,Austin County,5,0,8,53,121,283,13,61,417,482,29354
1,Austin County,5,0,6,43,136,239,31,49,406,466,29718
2,Austin County,5,0,11,34,98,183,45,45,326,376,29963
3,Austin County,5,0,11,32,82,138,29,43,249,295,29912
4,Austin County,5,0,5,38,80,147,29,43,256,302,30009


### Split the Data into Training and Testing

In [6]:
# Create dummies
df = pd.get_dummies(df.drop(columns='County'))
df.head()

,Agency_Count,Murder,Rape,Assault,Burglary,Larceny,Auto_Theft,Violent_Offenses,NonViolent_Offenses,Total_Crime,Population
0,5,0,8,53,121,283,13,61,417,482,29354
1,5,0,6,43,136,239,31,49,406,466,29718
2,5,0,11,34,98,183,45,45,326,376,29963
3,5,0,11,32,82,138,29,43,249,295,29912
4,5,0,5,38,80,147,29,43,256,302,30009


In [7]:
# Create target
y = pd.DataFrame(df["Population"])

# Create features
X = pd.DataFrame(df[['Agency_Count','Total_Crime']])

In [8]:
X.describe()

,Agency_Count,Total_Crime
count,102.000000,102.000000
mean,8.617647,14348.470588
std,10.573687,45215.295287
min,1.000000,281.000000
25%,3.000000,820.750000
50%,4.000000,1166.000000
75%,11.000000,6549.500000
max,45.000000,197686.000000


In [9]:
y.head()

,Population
0,29354
1,29718
2,29963
3,29912
4,30009


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.shape

(76, 2)

## Ensemble Learners

In [11]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

brfc_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brfc_model.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [12]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = brfc_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.0

In [13]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [14]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      27379       0.00      0.00      1.00      0.00      0.00      0.00       1.0
      27614       0.00      0.00      0.96      0.00      0.00      0.00       0.0
      27822       0.00      0.00      1.00      0.00      0.00      0.00       1.0
      28219       0.00      0.00      1.00      0.00      0.00      0.00       1.0
      28406       0.00      0.00      1.00      0.00      0.00      0.00       1.0
      29045       0.00      0.00      0.88      0.00      0.00      0.00       0.0
      34688       0.00      0.00      0.96      0.00      0.00      0.00       0.0
      35066       0.00      0.00      1.00      0.00      0.00      0.00       1.0
      35139       0.00      0.00      0.96      0.00      0.00      0.00       0.0
      36010       0.00      0.00      1.00      0.00      0.00      0.00       1.0
      36674       0.00      0.00      1.00      0.00      0.00      0.00       1.0
   

In [15]:
# List the features sorted in descending order by feature importance
priority_features = pd.Series(data=brfc_model.feature_importances_,index=X.columns)
priority_features.sort_values(ascending=False,inplace=True)
print(priority_features)

Total_Crime     0.742274
Agency_Count    0.257726
dtype: float64


## Easy Ensemble AdaBoost Classifier

In [16]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier 

eec_model = EasyEnsembleClassifier(n_estimators =100, random_state=1)
eec_model.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [17]:
# Calculated the balanced accuracy score
y_pred = eec_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.0

In [18]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int64)

In [19]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      27379       0.00      0.00      1.00      0.00      0.00      0.00       1.0
      27614       0.00      0.00      0.96      0.00      0.00      0.00       0.0
      27822       0.00      0.00      1.00      0.00      0.00      0.00       1.0
      28219       0.00      0.00      1.00      0.00      0.00      0.00       1.0
      28406       0.00      0.00      1.00      0.00      0.00      0.00       1.0
      29045       0.00      0.00      0.92      0.00      0.00      0.00       0.0
      30009       0.00      0.00      0.96      0.00      0.00      0.00       0.0
      34688       0.00      0.00      0.96      0.00      0.00      0.00       0.0
      35066       0.00      0.00      1.00      0.00      0.00      0.00       1.0
      35139       0.00      0.00      0.96      0.00      0.00      0.00       0.0
      36010       0.00      0.00      1.00      0.00      0.00      0.00       1.0
   